In [ ]:
from typing import List, Optional, Tuple

import numpy as np

from l5kit.data import (
    TL_FACE_DTYPE,
    filter_agents_by_labels,
    filter_tl_faces_by_frames,
    get_agents_slice_from_frames,
    get_tl_faces_slice_from_frames,
)
from l5kit.data.filter import filter_agents_by_frames, filter_agents_by_track_id
from l5kit.geometry import rotation33_as_yaw, world_to_image_pixels_matrix
from l5kit.kinematic import Perturbation
from l5kit.rasterization import EGO_EXTENT_HEIGHT, EGO_EXTENT_LENGTH, EGO_EXTENT_WIDTH, Rasterizer
from l5kit.sampling.slicing import get_future_slice, get_history_slice


def generate_kinetic_agent_sample(
    state_index: int,
    frames: np.ndarray,
    agents: np.ndarray,
    tl_faces: np.ndarray,
    selected_track_id: Optional[int],
    raster_size: Tuple[int, int],
    pixel_size: np.ndarray,
    ego_center: np.ndarray,
    history_num_frames: int,
    history_step_size: int,
    future_num_frames: int,
    future_step_size: int,
    filter_agents_threshold: float,
    rasterizer: Optional[Rasterizer] = None,
    perturbation: Optional[Perturbation] = None,
) -> dict:
    """Generates the inputs and targets to train a deep prediction model. A deep prediction model takes as input
    the state of the world (here: an image we will call the "raster"), and outputs where that agent will be some
    seconds into the future.

    This function has a lot of arguments and is intended for internal use, you should try to use higher level classes
    and partials that use this function.

    Args:
        state_index (int): The anchor frame index, i.e. the "current" timestep in the scene
        frames (np.ndarray): The scene frames array, can be numpy array or a zarr array
        agents (np.ndarray): The full agents array, can be numpy array or a zarr array
        tl_faces (np.ndarray): The full traffic light faces array, can be numpy array or a zarr array
        selected_track_id (Optional[int]): Either None for AV, or the ID of an agent that you want to
        predict the future of. This agent is centered in the raster and the returned targets are derived from
        their future states.
        raster_size (Tuple[int, int]): Desired output raster dimensions
        pixel_size (np.ndarray): Size of one pixel in the real world
        ego_center (np.ndarray): Where in the raster to draw the ego, [0.5,0.5] would be the center
        history_num_frames (int): Amount of history frames to draw into the rasters
        history_step_size (int): Steps to take between frames, can be used to subsample history frames
        future_num_frames (int): Amount of history frames to draw into the rasters
        future_step_size (int): Steps to take between targets into the future
        filter_agents_threshold (float): Value between 0 and 1 to use as cutoff value for agent filtering
        based on their probability of being a relevant agent
        rasterizer (Optional[Rasterizer]): Rasterizer of some sort that draws a map image
        perturbation (Optional[Perturbation]): Object that perturbs the input and targets, used
to train models that can recover from slight divergence from training set data

    Raises:
        ValueError: A ValueError is returned if the specified ``selected_track_id`` is not present in the scene
        or was filtered by applying the ``filter_agent_threshold`` probability filtering.

    Returns:
        dict: a dict object with the raster array, the future offset coordinates (meters),
        the future yaw angular offset, the future_availability as a binary mask
    """

    agent_velocity = None

    #  the history slice is ordered starting from the latest frame and goes backward in time., ex. slice(100, 91, -2)
    history_slice = get_history_slice(state_index, history_num_frames, history_step_size, include_current_state=True)
    future_slice = get_future_slice(state_index, future_num_frames, future_step_size)

    history_frames = frames[history_slice].copy()  # copy() required if the object is a np.ndarray
    future_frames = frames[future_slice].copy()

    sorted_frames = np.concatenate((history_frames[::-1], future_frames))  # from past to future

    # get agents (past and future)
    agent_slice = get_agents_slice_from_frames(sorted_frames[0], sorted_frames[-1])
    agents = agents[agent_slice].copy()  # this is the minimum slice of agents we need
    history_frames["agent_index_interval"] -= agent_slice.start  # sync interval with the agents array
    future_frames["agent_index_interval"] -= agent_slice.start  # sync interval with the agents array
    history_agents = filter_agents_by_frames(history_frames, agents)
    future_agents = filter_agents_by_frames(future_frames, agents)

    try:
        tl_slice = get_tl_faces_slice_from_frames(history_frames[-1], history_frames[0])  # -1 is the farthest
        # sync interval with the traffic light faces array
        history_frames["traffic_light_faces_index_interval"] -= tl_slice.start
        history_tl_faces = filter_tl_faces_by_frames(history_frames, tl_faces[tl_slice].copy())
    except ValueError:
        history_tl_faces = [np.empty(0, dtype=TL_FACE_DTYPE) for _ in history_frames]

    if perturbation is not None:
        history_frames, future_frames = perturbation.perturb(
            history_frames=history_frames, future_frames=future_frames
        )

    # State you want to predict the future of.
    cur_frame = history_frames[0]
    cur_agents = history_agents[0]

    if selected_track_id is None:
        agent_centroid = cur_frame["ego_translation"][:2]

        # FIXME
        # Assume velocity was 0, but it probably shouldnt be
        agent_velocity = np.array([0, 0], dtype=np.float32)

        agent_yaw = rotation33_as_yaw(cur_frame["ego_rotation"])
        agent_extent = np.asarray((EGO_EXTENT_LENGTH, EGO_EXTENT_WIDTH, EGO_EXTENT_HEIGHT))
        selected_agent = None
    else:
        # this will raise IndexError if the agent is not in the frame or under agent-threshold
        # this is a strict error, we cannot recover from this situation
        try:
            agent = filter_agents_by_track_id(
                filter_agents_by_labels(cur_agents, filter_agents_threshold), selected_track_id
            )[0]
        except IndexError:
            raise ValueError(f" track_id {selected_track_id} not in frame or below threshold")
        agent_centroid = agent["centroid"]
        agent_velocity = agent["velocity"]
        agent_yaw = float(agent["yaw"])
        agent_extent = agent["extent"]
        selected_agent = agent

    input_im = (
        None
        if not rasterizer
        else rasterizer.rasterize(history_frames, history_agents, history_tl_faces, selected_agent)
    )

    world_to_image_space = world_to_image_pixels_matrix(
        raster_size,
        pixel_size,
        ego_translation_m=agent_centroid,
        ego_yaw_rad=agent_yaw,
        ego_center_in_image_ratio=ego_center,
    )

    future_coords_offset, future_vel_offset, future_accel_offset, _, future_yaws_offset, future_availability = _create_targets_for_deep_prediction(
        future_num_frames, future_frames, selected_track_id, future_agents, agent_centroid[:2], agent_velocity[:2],
        agent_yaw,
    )

    # history_num_frames + 1 because it also includes the current frame
    history_coords_offset, history_vel_offset, history_accel_offset, history_jerk_offset, history_yaws_offset, \
    history_availability = _create_targets_for_deep_prediction(history_num_frames + 1, history_frames,
        selected_track_id, history_agents, agent_centroid[:2], agent_velocity[:2], agent_yaw,
    )

    # estimated_future_positions = history_jerk_offset
    estimated_future_positions = _extrapolate_targets(future_num_frames, history_num_frames, history_coords_offset,
                                                      history_vel_offset, history_accel_offset, history_jerk_offset)

    error_arr = np.abs(estimated_future_positions - future_coords_offset[10:])
    error = np.average(error_arr)

    return {
        "image": input_im,
        "target_positions": future_coords_offset,
        "target_velocities": future_vel_offset,
        "target_accelerations": future_accel_offset,
        "target_yaws": future_yaws_offset,
        "target_availabilities": future_availability,
        "history_positions": history_coords_offset,
        "history_velocities": history_vel_offset,
        "history_accelerations": history_accel_offset,
        "history_yaws": history_yaws_offset,
        "history_availabilities": history_availability,
        "estimated_future_positions": estimated_future_positions,
        "world_to_image": world_to_image_space,
        "centroid": agent_centroid,
        "yaw": agent_yaw,
        "extent": agent_extent,
    }


def _extrapolate_targets(future_num_frames: int, history_num_frames: int, position_history: np.ndarray,
                         vel_history: np.ndarray, accel_history: np.ndarray, jerk_history: np.ndarray) -> np.ndarray:
    """
    Extrapolates future positions using the first, second, and third derivatives of position

    hehe not really extrapolation, but I'm just doing some estimation atm

    :param future_num_frames:
    :param position_history:
    :param vel_history:
    :param accel_history:
    :param jerk_history:
    :return: Extrapolated future positions - (future-history x 2)
    """

    dt = 0.1

    start_time = len(position_history) * dt

    predict_frames = future_num_frames - history_num_frames

    time_arr = np.arange(dt, predict_frames * dt + dt, dt)

    avg_vel = np.array([np.average(vel_history.T[0]), np.average(vel_history.T[1])])
    avg_acc = np.array([np.average(accel_history.T[0]), np.average(accel_history.T[1])])
    avg_jrk = np.array([np.average(jerk_history.T[0]), np.average(jerk_history.T[1])])

    pos = np.array([position_history[-1]]).T
    vel = np.array([avg_vel]).T
    accel = np.array([avg_acc]).T
    jerk = np.array([avg_jrk]).T
    # vel = np.array([vel_history[-1]]).T
    # accel = np.array([accel_history[-1]]).T
    # jerk = np.array([jerk_history[-1]]).T

    # s =  	s0 + v0t + ½a0t2 + ⅙jt3
    res = pos * np.ones(time_arr.shape) + vel * time_arr + (accel * time_arr ** 2)/2 + (jerk * time_arr ** 3) / 6

    return res.T




def _create_targets_for_deep_prediction(
    num_frames: int,
    frames: np.ndarray,
    selected_track_id: Optional[int],
    agents: List[np.ndarray],
    agent_current_centroid: np.ndarray,
    agent_current_velocity: np.ndarray,
    agent_current_yaw: float,
) -> Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
    """
    Internal function that creates the targets and availability masks for deep prediction-type models.
    The futures/history offset (in meters) are computed. When no info is available (e.g. agent not in frame)
    a 0 is set in the availability array (1 otherwise).

    Args:
        num_frames (int): number of offset we want in the future/history
        frames (np.ndarray): available frames. This may be less than num_frames
        selected_track_id (Optional[int]): agent track_id or AV (None)
        agents (List[np.ndarray]): list of agents arrays (same len of frames)
        agent_current_centroid (np.ndarray): centroid of the agent at timestep 0
        agent_current_yaw (float): angle of the agent at timestep 0

    Returns:
        Tuple[np.ndarray, np.ndarray, np.ndarray]: position offsets, angle offsets, availabilities

    """
    #TODO: JERK

    # Inclusively count the number of available and unavailable frames of this target
    num_avail = 0
    num_unavail = 0

    last_avail_index = -1
    last_unavail_index = -1

    # How much the coordinates differ from the current state in meters.
    coords_offset = np.zeros((num_frames, 2), dtype=np.float32)
    vel_offset = np.zeros((num_frames, 2), dtype=np.float32)
    accel_offset = np.zeros((num_frames, 2), dtype=np.float32)
    jerk_offset = np.zeros((num_frames, 2), dtype=np.float32)
    yaws_offset = np.zeros((num_frames, 1), dtype=np.float32)
    availability = np.zeros((num_frames,), dtype=np.float32)

    for i, (frame, agents) in enumerate(zip(frames, agents)):
        if selected_track_id is None:
            agent_centroid = frame["ego_translation"][:2]
            agent_yaw = rotation33_as_yaw(frame["ego_rotation"])
            agent_velocity = np.array([0, 0], dtype=np.float32)

            num_avail = num_avail + 1
        else:
            # it's not guaranteed the target will be in every frame
            try:
                agent = filter_agents_by_track_id(agents, selected_track_id)[0]
                num_avail = num_avail + 1
            except IndexError:
                availability[i] = 0.0  # keep track of invalid futures/history
                last_unavail_index = i
                num_unavail = num_unavail + 1
                continue

            agent_centroid = agent["centroid"]
            agent_velocity = agent["velocity"]
            agent_yaw = agent["yaw"]

        #TODO: change my kinetics to the format of
        # offset for time[i] - time[0]
        # So tidy up the velocity calculation process, and the acceleration stuff
        # time 0 state, and time > 0 state. and make the calculations with respect to time passed and the time 0 val
        # Also provide a means of getting change in time

        # Note: B is an initial case
        # State A: the boy scout case. All clean, no trouble
        #   Two or more positions, previous frame is available
        # State B: One or none positions
        # State C: Two or more positions, previous frame is unavail

        coords_offset[i] = agent_centroid - agent_current_centroid

        # STATE B
        if num_avail == 1:
            # There is only the current position, no velocity is possible
            vel_offset[i] = agent_current_velocity
            accel_offset[i] = np.array([0, 0], dtype=np.float32)
            jerk_offset[i] = np.array([0, 0], dtype=np.float32)

        # there are two or more positions
        # There has been no unavailabilities yet
        elif last_unavail_index == -1:
            # STATE A
            # Only 1 frame difference in time
            dx = coords_offset[i] - coords_offset[i - 1]
            dt = 0.1

            vel_offset[i] = dx / dt  # change in position / frame width (0.1 seconds) : [m/s]
            # accel_offset[i] = (agent_velocity ** 2 - vel_offset[i - 1] ** 2) / (2 * dx)
            accel_offset[i] = (vel_offset[i] - vel_offset[i - 1]) / dt
            jerk_offset[i] = (accel_offset[i] - accel_offset[i - 1]) / dt

        # Unavailability was previous frame
        elif last_unavail_index == i - 1:
            # Check number of positions
            # STATE C
            # There are positions before the unavailability, calculate the velocity
            # Get previous existing velocity
            prev_vel = vel_offset[last_avail_index]

            # Get previous existing position
            prev_pos = coords_offset[last_avail_index]

            # Current Position
            pos = coords_offset[i]

            # Delta X
            dx = pos - prev_pos

            # Delta Time...0.1 being the frame width
            dt = (i - last_avail_index) * 0.1

            # Calculate velocity
            vel = (dx / dt) * - prev_vel
            vel_offset[i] = vel

            # Now calculate acceleration
            accel_offset[i] = (((dx - prev_vel * dt) * 2) ** (1/2)) / dt

            # Now Jerk
            jerk_offset[i] = (accel_offset[i] - accel_offset[last_avail_index]) / dt

        else:
            # STATE A
            # Only 1 frame difference in time
            dx = coords_offset[i] - coords_offset[i - 1]
            dt = 0.1

            vel_offset[i] = dx / dt  # change in position / frame width (0.1 seconds) : [m/s]
            accel_offset[i] = (agent_velocity ** 2 - vel_offset[i - 1] ** 2) / (2 * dx)
            jerk_offset[i] = (accel_offset[i] - accel_offset[i - 1]) / dt
        # else:
        #
        #     # Attempt to get previous velocity
        #     if i == 0:
        #         # No previous velocity
        #         prev_vel = np.array([0, 0], dtype=np.float32)
        #     else:
        #         # There must be a previous velocity
        #         prev_vel = vel_offset[i - 1]
        #
        #     # Calculate the acceleration
        #     accel_offset[i] = (vel_offset[i] ** 2 - prev_vel ** 2) / (2 * coords_offset[i])
        # else:
        #     accel_offset[i] = (agent_velocity ** 2 - agent_current_velocity ** 2) / (2 * coords_offset[i])

        yaws_offset[i] = agent_yaw - agent_current_yaw
        availability[i] = 1.0

        # save this index as the last available index
        last_avail_index = i
    return coords_offset, vel_offset, accel_offset, jerk_offset, yaws_offset, availability

In [ ]:
from l5kit.dataset.ego import EgoDataset
from l5kit.dataset.agent import AgentDataset
from pathlib import Path
import bisect

from l5kit.data import ChunkedDataset, get_agents_slice_from_frames, get_frames_slice_from_scenes
from l5kit.kinematic import Perturbation
from l5kit.rasterization import Rasterizer
from l5kit.dataset.select_agents import TH_DISTANCE_AV, TH_EXTENT_RATIO, TH_YAW_DEGREE, select_agents

import numpy as np
from zarr import convenience
from functools import partial
from typing import Optional, Tuple, cast


# WARNING: changing these values impact the number of instances selected for both train and inference!
MIN_FRAME_HISTORY = 10  # minimum number of frames an agents must have in the past to be picked
MIN_FRAME_FUTURE = 1  # minimum number of frames an agents must have in the future to be picked


class KineticDataset(AgentDataset):

    def __init__(
        self,
        cfg: dict,
        zarr_dataset: ChunkedDataset,
        rasterizer: Rasterizer,
        perturbation: Optional[Perturbation] = None,
        agents_mask: Optional[np.ndarray] = None,
        min_frame_history: int = MIN_FRAME_HISTORY,
        min_frame_future: int = MIN_FRAME_FUTURE,
    ):
        assert perturbation is None, "KineticDataset does not support perturbation (yet)"

        super(KineticDataset, self).__init__(cfg, zarr_dataset, rasterizer, perturbation, agents_mask,
                                             min_frame_history, min_frame_future)

        # build a partial so we don't have to access cfg each time
        self.sample_function = partial(
            generate_kinetic_agent_sample,
            raster_size=cast(Tuple[int, int], tuple(cfg["raster_params"]["raster_size"])),
            pixel_size=np.array(cfg["raster_params"]["pixel_size"]),
            ego_center=np.array(cfg["raster_params"]["ego_center"]),
            history_num_frames=cfg["model_params"]["history_num_frames"],
            history_step_size=cfg["model_params"]["history_step_size"],
            future_num_frames=cfg["model_params"]["future_num_frames"],
            future_step_size=cfg["model_params"]["future_step_size"],
            filter_agents_threshold=cfg["raster_params"]["filter_agents_threshold"],
            rasterizer=rasterizer,
            perturbation=perturbation,
        )

    def get_frame(self, scene_index: int, state_index: int, track_id: Optional[int] = None) -> dict:
        """
        A utility function to get the rasterisation and trajectory target for a given agent in a given frame

        Args:
            scene_index (int): the index of the scene in the zarr
            state_index (int): a relative frame index in the scene
            track_id (Optional[int]): the agent to rasterize or None for the AV
        Returns:
            dict: the rasterised image, the target trajectory (position and yaw) along with their availability,
            the 2D matrix to center that agent, the agent track (-1 if ego) and the timestamp

        """
        frames = self.dataset.frames[get_frames_slice_from_scenes(self.dataset.scenes[scene_index])]
        data = self.sample_function(state_index, frames, self.dataset.agents, self.dataset.tl_faces, track_id)
        # 0,1,C -> C,0,1
        image = data["image"].transpose(2, 0, 1)

        target_positions = np.array(data["target_positions"], dtype=np.float32)
        # target_velocities = np.array(data["target_velocities"], dtype=np.float32)
        # target_accelerations = np.array(data["target_accelerations"], dtype=np.float32)
        target_yaws = np.array(data["target_yaws"], dtype=np.float32)

        history_positions = np.array(data["history_positions"], dtype=np.float32)
        # history_velocities = np.array(data["history_velocities"], dtype=np.float32)
        # history_accelerations = np.array(data["history_accelerations"], dtype=np.float32)
        # history_yaws = np.array(data["history_yaws"], dtype=np.float32)

        estimated_positions = np.array(data["estimated_future_positions"], dtype=np.float32)

        timestamp = frames[state_index]["timestamp"]
        track_id = np.int64(-1 if track_id is None else track_id)  # always a number to avoid crashing torch

        return {
            "image": image,
            "target_positions": target_positions,
            "target_yaws": target_yaws,
            "target_availabilities": data["target_availabilities"],
            "history_positions": history_positions,
            "history_availabilities": data["history_availabilities"],
            "estimated_future_positions": estimated_positions,
            "world_to_image": data["world_to_image"],
            "track_id": track_id,
            "timestamp": timestamp,
            "centroid": data["centroid"],
            "yaw": data["yaw"],
            "extent": data["extent"],
        }

In [ ]:
import torch
import torch.nn as nn
from torchvision.models import mnasnet1_0


def trim_network_at_index(network: nn.Module, index: int = -1) -> nn.Module:
    """
    Returns a new network with all layers up to index from the back.
    :param network: Module to trim.
    :param index: Where to trim the network. Counted from the last layer.
    """
    assert index < 0, f"Param index must be negative. Received {index}."
    return nn.Sequential(*list(network.children())[:index])


class MnasBackbone(nn.Module):
    """
    Outputs tensor after last convolution before the fully connected layer.

    Allowed versions: mobilenet_v2.
    """

    def __init__(self, num_in_channels: int = 3):
        """
        Inits MobileNetBackbone.
        :param version: mobilenet version to use.
        """
        super().__init__()

        model = mnasnet1_0(pretrained=False)
        layer = model.layers[0]
        model.layers[0] = nn.Conv2d(
            in_channels=num_in_channels,
            out_channels=layer.out_channels,
            kernel_size=layer.kernel_size,
            stride=layer.stride,
            padding=layer.padding,
            bias=False,
        )

        self.backbone = trim_network_at_index(model, -1)

    def forward(self, input_tensor: torch.Tensor) -> torch.Tensor:
        """
        Outputs features after last convolution.
        :param input_tensor:  Shape [batch_size, n_channels, length, width].
        :return: Tensor of shape [batch_size, n_convolution_filters]. For mobilenet_v2,
            the shape is [batch_size, 1280].
        """
        backbone_features = self.backbone(input_tensor)
        return backbone_features.mean([2, 3])



In [ ]:
import math
import random
from typing import List, Tuple

import torch
from torch import nn
from torch.nn import functional as f

# Number of entries in Agent State Vector
ASV_DIM = 3

def calculate_backbone_feature_dim(backbone, input_shape: Tuple[int, int, int]) -> int:
    """ Helper to calculate the shape of the fully-connected regression layer. """
    tensor = torch.ones(1, *input_shape)
    output_feat = backbone.forward(tensor)
    return output_feat.shape[-1]

class LyftNet(nn.Module):
    """
    Implementation of Multiple-Trajectory Prediction (MTP) model
    based on https://arxiv.org/pdf/1809.10732.pdf
    """

    def __init__(self, backbone: nn.Module, num_modes: int, num_targets: int, num_kinetic_dim: int = 3,
                 n_hidden_layers: int = 4096, input_shape: Tuple[int, int, int] = (3, 500, 500)):
        """
        Inits the MTP network.
        :param backbone: CNN Backbone to use.
        :param num_modes: Number of predicted paths to estimate for each agent.
        :param seconds: Number of seconds into the future to predict.
            Default for the challenge is 6.
        :param frequency_in_hz: Frequency between timesteps in the prediction (in Hz).
            Highest frequency is nuScenes is 2 Hz.
        :param n_hidden_layers: Size of fully connected layer after the CNN
            backbone processes the image.
        :param input_shape: Shape of the input expected by the network.
            This is needed because the size of the fully connected layer after
            the backbone depends on the backbone and its version.
        Note:
            Although seconds and frequency_in_hz are typed as floats, their
            product should be an int.
        """

        super().__init__()

        self.ASV_DIM = num_kinetic_dim

        self.backbone = backbone
        self.num_modes = num_modes
        backbone_feature_dim = calculate_backbone_feature_dim(backbone, input_shape)
        self.fc1 = nn.Linear(backbone_feature_dim + self.ASV_DIM, n_hidden_layers, bias=True)
        # predictions_per_mode = int(seconds * frequency_in_hz) * 2
        predictions_per_mode = num_targets

        self.num_pred = predictions_per_mode * num_modes
        self.future_len = int(num_targets / 2)

        self.norm = nn.BatchNorm1d(self.ASV_DIM)

        self.fc2 = nn.Linear(n_hidden_layers, int(self.num_pred + num_modes))

    def forward(self, image_tensor: torch.Tensor,
                agent_state_vector: torch.Tensor) -> torch.Tensor:
        """
        Forward pass of the model.
        :param image_tensor: Tensor of images shape [batch_size, n_channels, length, width].
        :param agent_state_vector: Tensor of floats representing the agent state.
            [batch_size, ASV_DIM * num_targets].
        :returns: Tensor of dimension [batch_size, number_of_modes * number_of_predictions_per_mode + number_of_modes]
            storing the predicted trajectory and mode probabilities. Mode probabilities are normalized to sum
            to 1 during inference.
        """

        backbone_features = self.backbone(image_tensor)

        features = torch.cat([backbone_features, self.norm(agent_state_vector)], dim=1)

        predictions = self.fc2(self.fc1(features))

        bs, _ = predictions.shape

        pred, confidences = torch.split(predictions, self.num_pred, dim=1)

        pred = pred.view(bs, self.num_modes, self.future_len, 2)
        assert confidences.shape == (bs, self.num_modes)

        confidences = torch.softmax(confidences, dim=1)

        # print(self.fc1.weight)
        # print("---------------")

        # nn.utils.clip_grad_norm_()

        if math.isnan(confidences[0][0].item()):
            print("Oh no, not the naan")
            print(agent_state_vector)

        return pred, confidences

In [ ]:
# --- Function utils ---
# Original code from https://github.com/lyft/l5kit/blob/20ab033c01610d711c3d36e1963ecec86e8b85b6/l5kit/l5kit/evaluation/metrics.py
import numpy as np

import torch
from torch import Tensor

def pytorch_neg_multi_log_likelihood_batch(
    gt: Tensor, pred: Tensor, confidences: Tensor, avails: Tensor
) -> Tensor:
    """
    Compute a negative log-likelihood for the multi-modal scenario.
    log-sum-exp trick is used here to avoid underflow and overflow, For more information about it see:
    https://en.wikipedia.org/wiki/LogSumExp#log-sum-exp_trick_for_log-domain_calculations
    https://timvieira.github.io/blog/post/2014/02/11/exp-normalize-trick/
    https://leimao.github.io/blog/LogSumExp/
    Args:
        gt (Tensor): array of shape (bs)x(time)x(2D coords)
        pred (Tensor): array of shape (bs)x(modes)x(time)x(2D coords)
        confidences (Tensor): array of shape (bs)x(modes) with a confidence for each mode in each sample
        avails (Tensor): array of shape (bs)x(time) with the availability for each gt timestep
    Returns:
        Tensor: negative log-likelihood for this example, a single float number
    """
    assert len(pred.shape) == 4, f"expected 3D (MxTxC) array for pred, got {pred.shape}"
    batch_size, num_modes, future_len, num_coords = pred.shape

    assert gt.shape == (batch_size, future_len, num_coords), f"expected 2D (Time x Coords) array for gt, got {gt.shape}"
    assert confidences.shape == (batch_size, num_modes), f"expected 1D (Modes) array for gt, got {confidences.shape}"
    if not torch.allclose(torch.sum(confidences, dim=1), confidences.new_ones((batch_size,))):
        print(confidences)
    assert torch.allclose(torch.sum(confidences, dim=1), confidences.new_ones((batch_size,))), "confidences should sum to 1"
    assert avails.shape == (batch_size, future_len), f"expected 1D (Time) array for gt, got {avails.shape}"
    # assert all data are valid
    assert torch.isfinite(pred).all(), "invalid value found in pred"
    assert torch.isfinite(gt).all(), "invalid value found in gt"
    assert torch.isfinite(confidences).all(), "invalid value found in confidences"
    assert torch.isfinite(avails).all(), "invalid value found in avails"

    # convert to (batch_size, num_modes, future_len, num_coords)
    gt = torch.unsqueeze(gt, 1)  # add modes
    avails = avails[:, None, :, None]  # add modes and cords

    # error (batch_size, num_modes, future_len)
    error = torch.sum(((gt - pred) * avails) ** 2, dim=-1)  # reduce coords and use availability

    with np.errstate(divide="ignore"):  # when confidence is 0 log goes to -inf, but we're fine with it
        # error (batch_size, num_modes)
        error = torch.log(confidences) - 0.5 * torch.sum(error, dim=-1)  # reduce time

    # use max aggregator on modes for numerical stability
    # error (batch_size, num_modes)
    max_value, _ = error.max(dim=1, keepdim=True)  # error are negative at this point, so max() gives the minimum one
    error = -torch.log(torch.sum(torch.exp(error - max_value), dim=-1, keepdim=True)) - max_value  # reduce modes
    # print("error", error)
    return torch.mean(error)


def pytorch_neg_multi_log_likelihood_single(
    gt: Tensor, pred: Tensor, avails: Tensor
) -> Tensor:
    """

    Args:
        gt (Tensor): array of shape (bs)x(time)x(2D coords)
        pred (Tensor): array of shape (bs)x(time)x(2D coords)
        avails (Tensor): array of shape (bs)x(time) with the availability for each gt timestep
    Returns:
        Tensor: negative log-likelihood for this example, a single float number
    """
    # pred (bs)x(time)x(2D coords) --> (bs)x(mode=1)x(time)x(2D coords)
    # create confidence (bs)x(mode=1)
    batch_size, future_len, num_coords = pred.shape
    confidences = pred.new_ones((batch_size, 1))
    return pytorch_neg_multi_log_likelihood_batch(gt, pred.unsqueeze(1), confidences, avails)

In [ ]:
from l5kit.evaluation import write_pred_csv

from l5kit.data import LocalDataManager, ChunkedDataset
from l5kit.rasterization import build_rasterizer
from l5kit.dataset import AgentDataset

from torch.utils.data import DataLoader

import os
import torch

from tqdm import tqdm
import torch.nn as nn
import torch.optim as optim
import numpy as np

import gc

import pandas as pd


class LyftManager:

    def __init__(self, config, data_path, device, num_modes=3, verbose=False):

        self.cfg = config
        self.data_path = data_path
        self.device = device

        self.verbose = verbose

        # change input channels number to match the rasterizer's output
        num_history_channels = (self.cfg["model_params"]["history_num_frames"] + 1) * 2
        num_in_channels = 3 + num_history_channels

        # Create backbone
        self.backbone = MnasBackbone(num_in_channels=num_in_channels)

        # Raster Size
        raster_size = self.cfg["raster_params"]["raster_size"]

        # change output size to (X, Y) * number of future states
        num_targets = 2 * self.cfg["model_params"]["future_num_frames"]
        self.future_len = self.cfg["model_params"]["future_num_frames"]
        print("Number of Targets = ", self.future_len)
        self.num_targets = num_targets
        self.num_preds = num_targets * num_modes
        self.num_modes = num_modes

        self.model = LyftNet(self.backbone, num_modes=num_modes, num_kinetic_dim=2 * (self.future_len + 1),
                             num_targets=num_targets, input_shape=(num_in_channels, raster_size[0], raster_size[1]))

        self.model.to(device=self.device)

    def train(self, iterations, lr=1e-3, file_name="lyft-net.pth"):

        # set env variable for data
        os.environ["L5KIT_DATA_FOLDER"] = self.data_path
        dm = LocalDataManager(None)
        # get config
        cfg = self.cfg
        print(cfg)

        # ===== INIT DATASET
        train_cfg = cfg["train_data_loader"]

        # Rasterizer
        rasterizer = build_rasterizer(cfg, dm)

        # Train dataset/dataloader
        train_zarr = ChunkedDataset(dm.require(train_cfg["key"])).open()
        if self.verbose:
            print("Dataset Chunked")
        train_dataset = KineticDataset(cfg, train_zarr, rasterizer)
        if self.verbose:
            print("Agent Dataset Retrieved")

        # agents = pd.DataFrame.from_records(train_zarr.agents,
        #                                    columns=['centroid', 'extent', 'yaw', 'velocity', 'track_id',
        #                                             'label_probabilities'])

        #
        # print(agents.head())

        train_dataloader = DataLoader(train_dataset,
                                      shuffle=train_cfg["shuffle"],
                                      batch_size=train_cfg["batch_size"],
                                      num_workers=train_cfg["num_workers"])

        print(train_dataset)

        # ==== INIT MODEL parameters
        optimizer = optim.Adam(self.model.parameters(), lr=lr)
        criterion = nn.PoissonNLLLoss()

        # ==== TRAIN LOOP

        tr_it = iter(train_dataloader)
        progress_bar = tqdm(range(iterations))
        losses_train = []
        rolling_avg = []
        # torch.save(model.state_dict(), "/home/michael/Workspace/Lyft/model/resnet_base.pth")
        for i in progress_bar:
            try:
                data = next(tr_it)
            except StopIteration:
                tr_it = iter(train_dataloader)
                data = next(tr_it)
            self.model.train()
            torch.set_grad_enabled(True)

            ####################
            # this is where it gets real funky

            # ids = data["track_id"]
            position_tensor = data["target_positions"].to(self.device)
            # velocity_tensor = data["target_velocities"].to(self.device)
            # acceleration_tensor = data["target_accelerations"].to(self.device)
            # yaw_tensor = data["target_yaws"].to(self.device)

            history_position_tensor = data["history_positions"].to(self.device)
            estimated_future_positions = data["estimated_future_positions"].to(self.device)
            # history_velocity_tensor = data["history_velocities"].to(self.device)
            # history_acceleration_tensor = data["history_accelerations"].to(self.device)
            # history_yaw_tensor = data["history_yaws"].to(self.device)
            # history_availability = data["history_availabilities"].to(self.device)

            imageTensor = data["image"].to(self.device)
            if self.verbose:
                print("Image Tensor: ", imageTensor.shape)

            # state_vector = torch.cat([history_position_tensor, history_velocity_tensor, history_acceleration_tensor,
            #                           history_yaw_tensor], 2).to(self.device)

            state_vector = torch.cat([estimated_future_positions, history_position_tensor], 1).to(self.device)

            state_vector = torch.flatten(state_vector, 1).to(self.device)
            if self.verbose:
                print("State Vector: ", state_vector.shape)

            pred, conf = self.model.forward(imageTensor, state_vector)

            # loss2 = criterion(pred, position_tensor)

            # print(loss2)

            if self.verbose:
                print("Prediction: ", pred.shape)
                print("Probability: ", conf.shape)

            target_availabilities = data["target_availabilities"].unsqueeze(-1).to(self.device)

            flattenedTargets = torch.flatten(target_availabilities, 1, 2)

            loss = pytorch_neg_multi_log_likelihood_batch(position_tensor, pred, conf, flattenedTargets)

            # loss = self.lossModel(pred, data["target_positions"])


            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            losses_train.append(loss.item())
            rolling_avg.append(np.mean(losses_train))
            progress_bar.set_description(f"loss: {loss.item()} loss(avg): {np.mean(losses_train)}")

            # if i == 10000:
            #     torch.save(model.state_dict(), "/home/michael/Workspace/Lyft/model/resnet" + str(i) + ".pth")

        print("Done Training")
        torch.save(self.model.state_dict(), f"/home/michael/Workspace/Lyft/model/{file_name}")

    def evaluate(self, data_path, file_name="submission.csv"):

        # set env variable for data
        os.environ["L5KIT_DATA_FOLDER"] = data_path
        dm = LocalDataManager(None)

        cfg = self.cfg

        # ===== INIT DATASET
        test_cfg = cfg["test_data_loader"]

        # Rasterizer
        rasterizer = build_rasterizer(cfg, dm)

        # Test dataset/dataloader
        test_zarr = ChunkedDataset(dm.require(test_cfg["key"])).open()
        test_mask = np.load(f"{data_path}/scenes/mask.npz")["arr_0"]
        # test_dataset = AgentDataset(cfg, test_zarr, rasterizer, agents_mask=test_mask)
        test_dataset = KineticDataset(cfg, test_zarr, rasterizer, agents_mask=test_mask)
        test_dataloader = DataLoader(test_dataset,
                                     shuffle=test_cfg["shuffle"],
                                     batch_size=test_cfg["batch_size"],
                                     num_workers=test_cfg["num_workers"])
        test_dataloader = test_dataloader
        print(test_dataloader)

        # ==== EVAL LOOP
        self.model.eval()
        torch.set_grad_enabled(False)
        criterion = nn.MSELoss(reduction="none")

        # store information for evaluation
        future_coords_offsets_pd = []
        timestamps = []
        pred_coords =  []
        confidences_list = []

        agent_ids = []
        progress_bar = tqdm(test_dataloader)
        for data in progress_bar:

            # ids = data["track_id"]
            # position_tensor = data["target_positions"].to(self.device)
            # velocity_tensor = data["target_velocities"].to(self.device)
            # acceleration_tensor = data["target_accelerations"].to(self.device)
            # yaw_tensor = data["target_yaws"].to(self.device)

            history_position_tensor = data["history_positions"].to(self.device)
            estimated_future_positions = data["estimated_future_positions"].to(self.device)
            # history_velocity_tensor = data["history_velocities"].to(self.device)
            # history_acceleration_tensor = data["history_accelerations"].to(self.device)
            # history_yaw_tensor = data["history_yaws"].to(self.device)
            # history_availability = data["history_availabilities"].to(self.device)

            imageTensor = data["image"].to(self.device)
            if self.verbose:
                print("Image Tensor: ", imageTensor.shape)

            # state_vector = torch.cat([history_position_tensor, history_velocity_tensor, history_acceleration_tensor,
            #                           history_yaw_tensor], 2).to(self.device)

            state_vector = torch.cat([estimated_future_positions, history_position_tensor], 1).to(self.device)

            state_vector = torch.flatten(state_vector, 1).to(self.device)
            # print(state_vector)
            if self.verbose:
                print("State Vector: ", state_vector.shape)

            pred, confidences = self.model.forward(imageTensor, state_vector)

            # future_coords_offsets_pd.append(outputs.cpu().numpy().copy())
            timestamps.append(data["timestamp"].numpy().copy())
            agent_ids.append(data["track_id"].numpy().copy())
            #
            # pred, confidences = predictor(image)

            pred_coords.append(pred.cpu().numpy().copy())
            confidences_list.append(confidences.cpu().numpy().copy())

        # ==== Save Results
        pred_path = f"{os.getcwd()}/{file_name}"
        write_pred_csv(pred_path,
                       timestamps=np.concatenate(timestamps),
                       track_ids=np.concatenate(agent_ids),
                       coords=np.concatenate(pred_coords),
                       confs=np.concatenate(confidences_list))

        torch.cuda.empty_cache()

    def load(self, checkpoint_path:str):

        state_dict = torch.load(checkpoint_path, map_location=self.device)
        self.model.load_state_dict(state_dict)

    # def clean(self):  # DOES WORK
    #     self._optimizer_to(torch.device('cuda:0'))
    #     del self.optimizer
    #     gc.collect()
    #     torch.cuda.empty_cache()
    #
    # def _optimizer_to(self, device):
    #     for param in self.optimizer.state.values():
    #         # Not sure there are any global tensors in the state dict
    #         if isinstance(param, torch.Tensor):
    #             param.data = param.data.to(device)
    #             if param._grad is not None:
    #                 param._grad.data = param._grad.data.to(device)
    #         elif isinstance(param, dict):
    #             for subparam in param.values():
    #                 if isinstance(subparam, torch.Tensor):
    #                     subparam.data = subparam.data.to(device)
    #                     if subparam._grad is not None:
    #                         subparam._grad.data = subparam._grad.data.to(device)



In [ ]:
import torch
from l5kit.configs import load_config_data

data_path = "/kaggle/input/lyft-motion-prediction-autonomous-vehicles"

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
config = load_config_data("/kaggle/input/lyftnet/config_lyftnet.yaml")

config["test_data_loader"]["batch_size"] = 16
config["test_data_loader"]["num_workers"] = 8

checkpoint_path = "/kaggle/input/lyftnet/lyft-net.pth"


manager = LyftManager(config=config, data_path=data_path, device=device, num_modes=3, verbose=False)

manager.load(checkpoint_path)

#toto train for 324000
# manager.train(iterations=44000, lr=1e-4, file_name="lyft-net.pth")
# manager.train(iterations=324000, lr=1e-4, file_name="lyft-net.pth")

manager.evaluate(data_path=data_path)

In [ ]:
# import os
# os.system("!cp /kaggle/input/lyftnet/submission.csv /kaggle/working/submission.csv")